# Hipótesis de trabajo

In [ ]:
# import libraries
import pandas as pd
from optimus import Optimus
op = Optimus()

In [ ]:
# Read the data
# Data from http://rpubs.com/rhuebner/HRCodebook-13
df = op.read.csv("data/hr-data.csv", header=True)

In [ ]:
# See the data
df.table()

In [ ]:
# Basic data cleaning
from pyspark.sql.functions import when, count, col, isnull


integer_cols = ["MaritalStatusID", "EmpStatusID", "DeptID", "PerfScoreID", "PositionID", "Termd", "ManagerID", 
                "EmpSatisfaction", "SpecialProjectsCount", "DaysLateLast30"]
float_cols = ["PayRate", "EngagementSurvey"]

for col_name in integer_cols:
    df = df.withColumn(col_name, col(col_name).cast('int'))
    
for col_name in float_cols:
    df = df.withColumn(col_name, col(col_name).cast('float'))
    
df = df.dropna(how="all")
df = df.cols.years_between("DOB", date_format="mm/dd/yy",output_cols="Age")

In [ ]:
import pyspark.sql.functions as F
from pyspark.sql import Window

window = Window.rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing)

def count_to_pct(df, columns):
    return (df.groupBy(*columns, "Termd")
            .agg(F.count("Termd").alias("count_with_churn"))
            .orderBy('count_with_churn', ascending=False)
            .filter(col("Termd") == 1)
            .groupBy(*columns)
            .agg(F.sum("count_with_churn").alias("count"))
            .withColumn('total',F.sum("count").over(window))
            .withColumn('pct',F.round(col('count')/col('total'),2))
            .drop("total"))

## ¿Es un problema departamental?

In [ ]:
df.groupby("Department").count().table()

In [ ]:
df.groupby("Department").sum("Termd").table()

In [ ]:
count_to_pct(df, columns=["Department"]).table()

## ¿Es un problema de estado civil?

In [ ]:
df.groupby("MaritalDesc").count().table()

In [ ]:
df.groupby("MaritalDesc").sum("Termd").table()

In [ ]:
count_to_pct(df, columns=["MaritalDesc"]).table()

## ¿Es un problema racial?

In [ ]:
df.groupby("RaceDesc").count().table()

In [ ]:
df.groupby("RaceDesc").sum("Termd").table()

In [ ]:
count_to_pct(df, columns=["RaceDesc"]).table()

## ¿Es un problema de performance?

In [ ]:
df.groupby("PerformanceScore").count().table()

In [ ]:
df.groupby("PerformanceScore").sum("Termd").table()

In [ ]:
count_to_pct(df, columns=["PerformanceScore"]).table()

## ¿Tendrá algo que ver con el manager?

In [ ]:
count_to_pct(df, columns=["ManagerName"]).table(21)

## ¿Es un problema de edad?

In [ ]:
from pyspark.ml.feature import Bucketizer
splits = [10, 20, 30, 40, 50, 60, float("inf")]

bucketizer = Bucketizer(splits=splits, inputCol="Age", outputCol="Age_bucket")
df = bucketizer.transform(df)

In [ ]:
df.groupby("Age_bucket").avg("Age").table()

In [ ]:
df.groupby("Age_bucket").count().table()

In [ ]:
df.groupby("Age_bucket").sum("Termd").table()

In [ ]:
count_to_pct(df, columns=["Age_bucket"]).table(21)

### Estudio de correlación

In [ ]:
df.plot.correlation(["Age","PayRate","Termd", "EmpSatisfaction", "SpecialProjectsCount", "EngagementSurvey"])